In [1]:
# import libs
import pandas as pd

In [2]:
retail = pd.read_csv("C:/Users/Ark/0_ANALYSIS/DA_projects/invoice_data/InvoiceNo_data.csv.zip", compression="zip", encoding = 'ISO-8859-1')

In [3]:
retail_columns = retail.columns

In [4]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## EDA

In [5]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
# any duplicates?
retail.duplicated().sum()

5268

In [7]:
retail = retail.drop_duplicates()

In [8]:
# Canceled transactions
retail.InvoiceNo.str.startswith('C').value_counts()

False    527390
True       9251
Name: InvoiceNo, dtype: int64

In [9]:
retail = retail.assign(canceled = retail.InvoiceNo.str.startswith('C').astype(int))

In [10]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,canceled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,0
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,0
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,0
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,0
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,0


In [11]:
# top 20% of Germany customers by num of InvoiceNo.
germany_top = retail\
    .query("Country == 'Germany'")\
    .groupby(['CustomerID'], as_index=False)\
    .agg({'InvoiceNo' : 'nunique'})
germany_top.head()

,CustomerID,InvoiceNo
0,12426.0,1
1,12427.0,5
2,12468.0,3
3,12471.0,49
4,12472.0,13


In [12]:
quant = germany_top.InvoiceNo.quantile(q=0.8)
quant

9.0

In [13]:
germany_top = germany_top.query('InvoiceNo > @quant').CustomerID
germany_top

3     12471.0
4     12472.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
37    12569.0
44    12600.0
52    12619.0
53    12621.0
57    12626.0
65    12647.0
70    12662.0
78    12705.0
79    12708.0
80    12709.0
82    12712.0
84    12720.0
Name: CustomerID, dtype: float64

In [14]:
top_retail_germany = retail\
    .query("CustomerID in @germany_top")
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,canceled
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany,0
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany,0
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany,0
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany,0
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany,0
...,...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany,0
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany,0
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany,0
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany,0


In [15]:
# bestseller stockcodes
top_retail_germany\
    .query("StockCode != 'POST'")\
    .groupby("StockCode", as_index=False)\
    .agg({"Quantity":"count"})\
    .sort_values("Quantity", ascending=False)

,StockCode,Quantity
409,22326,62
452,22423,55
411,22328,45
474,22467,37
316,22077,34
...,...,...
359,22181,1
336,22128,1
139,21257,1
141,21259,1


In [16]:
# Column with revenue
retail = retail.assign(revenue = retail.Quantity * retail.UnitPrice)

In [17]:
retail.revenue.sum()

9726006.953999998

In [18]:
# Revenue of top-5 InvoiceNo
top_salers = retail\
    .groupby('InvoiceNo', as_index=False)\
    .agg({'revenue':'sum'})\
    .sort_values('revenue', ascending=False)\
    .head(5)

top_salers

,InvoiceNo,revenue
22025,581483,168469.60
2303,541431,77183.60
18776,574941,52940.94
19468,576365,50653.91
9741,556444,38970.00
